<a href="https://colab.research.google.com/github/BEMiklos/PlantTraits2024/blob/main/XGB_and_TabTransformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IMPORTS

In [19]:
!pip install -q kaggle

In [2]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
!kaggle competitions download -c planttraits2024

100% 3.17G/3.17G [02:13<00:00, 28.9MB/s]
100% 3.17G/3.17G [02:13<00:00, 25.5MB/s]


In [4]:
!unzip -q '/content/planttraits2024.zip'

In [2]:
import random

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import albumentations as A
import cv2

import torch
from torch import nn
import torch.nn.functional as F
from torchvision.io import read_image
from torch.utils.data import Dataset, ConcatDataset, DataLoader, Subset, random_split
from torchvision import transforms

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import r2_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import GridSearchCV

import xgboost as xgb

import re

In [3]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
random_state = 7

np.random.seed(0)

random.seed(0)

torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [4]:
TRAIN_IMAGES_PATH = '/content/train_images'
TEST_IMAGES_PATH = '/content/test_images'

### Untility functions

In [5]:
mean_pattern = r'X\d+_mean'
sd_pattern = r'X\d+_sd'

In [6]:
def find_matching_elements(lst,pattern):
    matching_elements = [element for element in lst if re.search(pattern, element)]
    return matching_elements

In [7]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [8]:
all_targets = [col for col in train_df.columns if col not in test_df.columns]
targets = all_targets[:6]

In [9]:
print(f'all targets: {all_targets}')
print(f'targets: {targets}')

all targets: ['X4_mean', 'X11_mean', 'X18_mean', 'X26_mean', 'X50_mean', 'X3112_mean', 'X4_sd', 'X11_sd', 'X18_sd', 'X26_sd', 'X50_sd', 'X3112_sd']
targets: ['X4_mean', 'X11_mean', 'X18_mean', 'X26_mean', 'X50_mean', 'X3112_mean']


In [10]:
metadata_columns = [col for col in train_df.columns if col in test_df.columns and col != 'id']

In [11]:
len(metadata_columns)

163

In [12]:
print(f'metadata columns: {metadata_columns}')

metadata columns: ['WORLDCLIM_BIO1_annual_mean_temperature', 'WORLDCLIM_BIO12_annual_precipitation', 'WORLDCLIM_BIO13.BIO14_delta_precipitation_of_wettest_and_dryest_month', 'WORLDCLIM_BIO15_precipitation_seasonality', 'WORLDCLIM_BIO4_temperature_seasonality', 'WORLDCLIM_BIO7_temperature_annual_range', 'SOIL_bdod_0.5cm_mean_0.01_deg', 'SOIL_bdod_100.200cm_mean_0.01_deg', 'SOIL_bdod_15.30cm_mean_0.01_deg', 'SOIL_bdod_30.60cm_mean_0.01_deg', 'SOIL_bdod_5.15cm_mean_0.01_deg', 'SOIL_bdod_60.100cm_mean_0.01_deg', 'SOIL_cec_0.5cm_mean_0.01_deg', 'SOIL_cec_100.200cm_mean_0.01_deg', 'SOIL_cec_15.30cm_mean_0.01_deg', 'SOIL_cec_30.60cm_mean_0.01_deg', 'SOIL_cec_5.15cm_mean_0.01_deg', 'SOIL_cec_60.100cm_mean_0.01_deg', 'SOIL_cfvo_0.5cm_mean_0.01_deg', 'SOIL_cfvo_100.200cm_mean_0.01_deg', 'SOIL_cfvo_15.30cm_mean_0.01_deg', 'SOIL_cfvo_30.60cm_mean_0.01_deg', 'SOIL_cfvo_5.15cm_mean_0.01_deg', 'SOIL_cfvo_60.100cm_mean_0.01_deg', 'SOIL_clay_0.5cm_mean_0.01_deg', 'SOIL_clay_100.200cm_mean_0.01_deg', 'S

In [13]:
train_df = train_df[train_df['X4_mean'] > -1]

In [14]:
targets

['X4_mean', 'X11_mean', 'X18_mean', 'X26_mean', 'X50_mean', 'X3112_mean']

In [15]:
  # Apply log10 transformation to the specified columns
for column in targets:
    mapped_values = train_df[column] + 1
    train_df[column] = np.log10(mapped_values)  # Adding 1 to avoid log(0)

<ipython-input-15-eb73a8baafcb>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df[column] = np.log10(mapped_values)  # Adding 1 to avoid log(0)


In [16]:
train_df

,id,WORLDCLIM_BIO1_annual_mean_temperature,WORLDCLIM_BIO12_annual_precipitation,WORLDCLIM_BIO13.BIO14_delta_precipitation_of_wettest_and_dryest_month,WORLDCLIM_BIO15_precipitation_seasonality,WORLDCLIM_BIO4_temperature_seasonality,WORLDCLIM_BIO7_temperature_annual_range,SOIL_bdod_0.5cm_mean_0.01_deg,SOIL_bdod_100.200cm_mean_0.01_deg,SOIL_bdod_15.30cm_mean_0.01_deg,...,X18_mean,X26_mean,X50_mean,X3112_mean,X4_sd,X11_sd,X18_sd,X26_sd,X50_sd,X3112_sd
0,192027691,12.235703,374.466675,62.524445,72.256844,773.592041,33.277779,125,149,136,...,0.048241,0.350980,0.454750,1.709406,0.008921,1.601473,0.025441,0.153608,0.279610,15.045054
1,195542235,17.270555,90.239998,10.351111,38.220940,859.193298,40.009777,124,144,138,...,0.142801,0.215622,0.371708,2.759742,0.003102,0.258078,0.000866,0.034630,0.010165,11.004477
2,196639184,14.254504,902.071411,49.642857,17.873655,387.977753,22.807142,107,133,119,...,0.980136,0.144649,0.524156,3.053500,NaN,NaN,NaN,NaN,NaN,NaN
3,195728812,18.680834,1473.933350,163.100006,45.009758,381.053986,20.436666,120,131,125,...,0.318820,0.062281,0.333509,3.018570,0.011692,2.818356,0.110673,0.011334,0.229224,141.857187
4,195251545,0.673204,530.088867,50.857777,38.230709,1323.526855,45.891998,91,146,120,...,0.219476,1.076275,0.511379,3.377937,0.006157,1.128000,0.026996,0.553815,0.107092,87.146899
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55484,190558785,19.472172,244.795914,39.127552,67.074493,472.710358,27.758673,118,140,131,...,0.091206,0.444543,0.416364,2.987017,NaN,NaN,NaN,NaN,NaN,NaN
55485,194523231,13.724150,1450.000000,162.260208,43.139324,652.716858,26.694387,125,144,135,...,0.304727,1.137134,0.533810,3.212458,0.005474,0.128133,0.117010,3.164520,0.082212,136.503697
55486,195888987,14.741204,581.866638,109.231110,89.272148,507.273010,26.874668,118,155,136,...,0.570239,1.049469,0.570846,2.780483,0.019727,0.215040,0.156309,0.919139,0.079395,26.159626
55487,135487319,16.094763,1180.838135,80.176193,22.909716,342.184021,17.346189,109,130,117,...,0.734773,1.015870,0.628567,2.389852,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
def split_dataframe(train_df, test_df, train_size=0.8, val_size=0.2):
    if train_size + val_size != 1:
        raise ValueError("train_size, val_size, and test_size must sum to 1")

    train_df, val_df = train_test_split(train_df, train_size=train_size, random_state=random_state)
    return train_df, val_df, test_df

In [18]:
def r2_loss(output, target):
    target = target
    output = output
    target_mean = torch.mean(target)
    ss_tot = torch.sum((target - target_mean) ** 2)
    ss_res = torch.sum((target - output) ** 2)
    r2 = 1 - ss_res / ss_tot
    return 1 - r2

# Data preparation

In [20]:
train_df, val_df, test_df = split_dataframe(train_df.sample(frac=1, random_state=1), test_df.sample(frac=1, random_state=1))

# Feature engineering

In [21]:
worldclim = train_df.columns[1:7]
soil = train_df.columns[7:68]
modis = train_df.columns[68:128]
vod = train_df.columns[128:164]

In [22]:
X_train, y_train = train_df[metadata_columns], train_df[targets]
X_val, y_val = val_df[metadata_columns], val_df[targets]
X_test = test_df[metadata_columns]

In [139]:
# scaler = StandardScaler()

# train_X_scaled = scaler.fit_transform(train_X_imputed)
# train_X_scaled = pd.DataFrame(train_X_scaled, columns=train_X_imputed.columns)

# test_X_scaled = scaler.fit_transform(test_X_imputed)
# test_X_scaled = pd.DataFrame(test_X_scaled, columns=test_X_imputed.columns)

# val_X_scaled = scaler.fit_transform(val_X_imputed)
# val_X_scaled = pd.DataFrame(val_X_scaled, columns=val_X_imputed.columns)

# **Training without scaling**

In [23]:
multi_xgb_model = MultiOutputRegressor(xgb.XGBRegressor(objective='reg:squarederror', n_estimators=200, random_state=42))

multi_xgb_model.fit(X_train, y_train)

MultiOutputRegressor(estimator=XGBRegressor(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None, device=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None,
                                            feature_types=None, gamma=None,
                                            grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=None, max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=None,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=200, n_jobs=None,
                                            num_parallel_tree=None,
                                            random_state=42, ...))

In [24]:
y_pred_multi = multi_xgb_model.predict(X_val)

In [25]:
r2_scores = r2_score(y_val[y_val.columns], y_pred_multi, multioutput='raw_values')
r2_scores = [f'{r2_score:.4f}' for r2_score in r2_scores]
print("R² scores for each target (on scaled features):", r2_scores)

R² scores for each target (on scaled features): ['0.1345', '0.1921', '0.3203', '0.2186', '0.0446', '0.2532']


In [26]:
multi_xgb_model = MultiOutputRegressor(xgb.XGBRegressor(objective='reg:squarederror', n_estimators=200, random_state=42))

multi_xgb_model.fit(X_train, y_train)

y_pred_multi = multi_xgb_model.predict(X_val)

r2_scores = r2_score(y_val, y_pred_multi, multioutput='raw_values')
r2_scores = [f'{r2_score:.4f}' for r2_score in r2_scores]
print("R² scores for each target (on scaled features):", r2_scores)

R² scores for each target (on scaled features): ['0.1345', '0.1921', '0.3203', '0.2186', '0.0446', '0.2532']


In [27]:
multi_xgb_model = MultiOutputRegressor(xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, random_state=42))

multi_xgb_model.fit(X_train, y_train)

y_pred_multi = multi_xgb_model.predict(X_val)

r2_scores = r2_score(y_val, y_pred_multi, multioutput='raw_values')
r2_scores = [f'{r2_score:.4f}' for r2_score in r2_scores]
print("R² scores for each target (on scaled features):", r2_scores)

R² scores for each target (on scaled features): ['0.1501', '0.2139', '0.3380', '0.2436', '0.0708', '0.2771']


# Optimalization trial - Dim reduction with PCA and selecting only the most important features

### Dimension reduction based on importance

In [28]:
xgb_model_full = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, random_state=42)
xgb_model_full.fit(X_train, y_train)

importances = xgb_model_full.feature_importances_

feature_names = X_train.columns
importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': importances})
importance_df = importance_df.sort_values(by='Importance', ascending=False).reset_index(drop=True)
print(importance_df.head(20))

                                              Feature  Importance
0              WORLDCLIM_BIO1_annual_mean_temperature    0.036271
1             WORLDCLIM_BIO7_temperature_annual_range    0.031863
2              WORLDCLIM_BIO4_temperature_seasonality    0.019308
3   MODIS_2000.2020_monthly_mean_surface_reflectan...    0.013995
4                    SOIL_sand_60.100cm_mean_0.01_deg    0.011784
5                   SOIL_phh2o_60.100cm_mean_0.01_deg    0.011778
6   MODIS_2000.2020_monthly_mean_surface_reflectan...    0.010594
7   MODIS_2000.2020_monthly_mean_surface_reflectan...    0.010476
8                     SOIL_ocd_60.100cm_mean_0.01_deg    0.009812
9                  VOD_C_2002_2018_multiyear_mean_m12    0.009591
10  WORLDCLIM_BIO13.BIO14_delta_precipitation_of_w...    0.009319
11                    SOIL_silt_30.60cm_mean_0.01_deg    0.009083
12                 VOD_C_2002_2018_multiyear_mean_m05    0.008678
13                 VOD_X_1997_2018_multiyear_mean_m09    0.008639
14  MODIS_

In [29]:
top_features = importance_df['Feature'].head(30).tolist()

multi_xgb_model = MultiOutputRegressor(xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, random_state=42))

multi_xgb_model.fit(X_train[top_features], y_train)

y_pred_multi = multi_xgb_model.predict(X_val[top_features])

r2_scores = r2_score(y_val, y_pred_multi, multioutput='raw_values')
r2_scores = [f'{r2_score:.4f}' for r2_score in r2_scores]
print("R² scores for each target (with the top 30 important features):", r2_scores)

R² scores for each target (with the top 30 important features): ['0.1466', '0.1973', '0.3465', '0.2501', '0.0737', '0.2787']


In [ ]:
[0.08292387 0.13063189 0.24397527 0.13770615 0.02839051 0.18928016]

# Dimension reduction with PCA

In [30]:
pca = PCA(n_components=min(X_train.shape))

pca.fit(X_train)

cumulative_variance_ratio = np.cumsum(pca.explained_variance_ratio_)

In [31]:
n_components_95 = np.where(cumulative_variance_ratio >= 0.95)[0][0] + 1

pca_95 = PCA(n_components=n_components_95)

train_pca_95 = pca_95.fit_transform(X_train)

train_pca_95_df = pd.DataFrame(train_pca_95, columns=[f'PC{i+1}' for i in range(n_components_95)])
train_pca_95_df.head(), n_components_95

(           PC1          PC2          PC3          PC4          PC5  \
 0 -3636.830789   744.590388  -786.504144  1424.625841  1020.546371   
 1 -1295.086190  1789.750797   506.264586  -910.384431 -1486.195806   
 2 -2455.309165  1454.214623   465.768375  -221.461044   733.301537   
 3  -309.863849  3524.609814 -1588.235186  -353.420764    99.065476   
 4 -2798.052048  1168.497457 -1471.376346   603.211178 -1128.095170   
 
            PC6         PC7  
 0  1053.110433 -191.917221  
 1  -699.980160  570.879727  
 2  -909.931057 -136.653272  
 3 -1006.795617 -321.104574  
 4  -320.774561  445.644513  ,
 7)

In [32]:
val_pca_95 = pca_95.fit_transform(X_val)

val_pca_95_df = pd.DataFrame(val_pca_95, columns=[f'PC{i+1}' for i in range(n_components_95)])
val_pca_95_df.head(), n_components_95

(            PC1           PC2          PC3          PC4          PC5  \
 0  20991.291110  -1972.543984 -1029.486667  1485.145637   689.805173   
 1  -3671.415846   -355.010320  -466.924569  1544.502853  -221.267257   
 2  -1865.741950   1991.986527 -1205.040895   148.168401 -1803.202510   
 3   7524.553906  10738.550423  1744.707691 -1953.116852  1433.935360   
 4  14992.288473  -2578.048967 -3834.244879 -3484.653659 -1081.027856   
 
           PC6         PC7  
 0  -37.437326  969.906639  
 1  413.750168  301.901406  
 2 -217.364239  688.693563  
 3 -783.035764  443.207949  
 4  433.696502  188.347513  ,
 7)

In [33]:
multi_xgb_model = MultiOutputRegressor(xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, random_state=42))

multi_xgb_model.fit(train_pca_95_df, y_train)

y_pred_multi = multi_xgb_model.predict(val_pca_95_df)

r2_scores = r2_score(y_val, y_pred_multi, multioutput='raw_values')
print("R² scores for each target (PCA):", r2_scores)

R² scores for each target (PCA): [0.08292387 0.13063189 0.24397527 0.13770615 0.02839051 0.18928016]


In [34]:
xgb_model_full = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, random_state=42)
xgb_model_full.fit(train_pca_95_df, y_train)

importances = xgb_model_full.feature_importances_

feature_names = train_pca_95_df.columns
train_importance_df_pca = pd.DataFrame({'Feature': feature_names, 'Importance': importances})
train_importance_df_pca = importance_df.sort_values(by='Importance', ascending=False).reset_index(drop=True)
print(train_importance_df_pca.head(20))

                                              Feature  Importance
0              WORLDCLIM_BIO1_annual_mean_temperature    0.036271
1             WORLDCLIM_BIO7_temperature_annual_range    0.031863
2              WORLDCLIM_BIO4_temperature_seasonality    0.019308
3   MODIS_2000.2020_monthly_mean_surface_reflectan...    0.013995
4                    SOIL_sand_60.100cm_mean_0.01_deg    0.011784
5                   SOIL_phh2o_60.100cm_mean_0.01_deg    0.011778
6   MODIS_2000.2020_monthly_mean_surface_reflectan...    0.010594
7   MODIS_2000.2020_monthly_mean_surface_reflectan...    0.010476
8                     SOIL_ocd_60.100cm_mean_0.01_deg    0.009812
9                  VOD_C_2002_2018_multiyear_mean_m12    0.009591
10  WORLDCLIM_BIO13.BIO14_delta_precipitation_of_w...    0.009319
11                    SOIL_silt_30.60cm_mean_0.01_deg    0.009083
12                 VOD_C_2002_2018_multiyear_mean_m05    0.008678
13                 VOD_X_1997_2018_multiyear_mean_m09    0.008639
14  MODIS_

In [36]:
results = []
for i in [10, 20, 30, 45, 60, 70, 80, 90, 100, 120, 130, 140, 150, 163]:
  train_top_features = train_importance_df_pca['Feature'].head(i).tolist()

  multi_xgb_model = MultiOutputRegressor(xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, random_state=42))

  multi_xgb_model.fit(X_train[train_top_features], y_train)

  y_pred_multi = multi_xgb_model.predict(X_val[train_top_features])

  r2_scores = r2_score(y_val, y_pred_multi, multioutput='raw_values')
  print(f"R² scores for each target (PCA with the top {i} important features):", r2_scores.mean())
  results.append(r2_scores.mean())


R² scores for each target (PCA with the top 10 important features): 0.2097206825069119
R² scores for each target (PCA with the top 20 important features): 0.21914703061915372
R² scores for each target (PCA with the top 30 important features): 0.21549137585044273
R² scores for each target (PCA with the top 45 important features): 0.21412244549054646
R² scores for each target (PCA with the top 60 important features): 0.21610426201551572
R² scores for each target (PCA with the top 70 important features): 0.2164570826188199
R² scores for each target (PCA with the top 80 important features): 0.21488398246279114
R² scores for each target (PCA with the top 90 important features): 0.21559051898447754
R² scores for each target (PCA with the top 100 important features): 0.21798512356783917
R² scores for each target (PCA with the top 120 important features): 0.21506310461452308
R² scores for each target (PCA with the top 130 important features): 0.21284945539429834
R² scores for each target (PCA 

In [41]:
train_top_features = train_importance_df_pca['Feature'].head(30).tolist()

multi_xgb_model = MultiOutputRegressor(xgb.XGBRegressor(objective='reg:squarederror', n_estimators=50, random_state=42))

multi_xgb_model.fit(X_train[train_top_features], y_train)

y_pred_multi = multi_xgb_model.predict(X_val[train_top_features])

r2_scores = r2_score(y_val, y_pred_multi, multioutput='raw_values')
print("R² scores for each target (PCA with the top 30 important features):", r2_scores)

R² scores for each target (PCA with the top 30 important features): [0.1529522  0.20510134 0.34704894 0.25620404 0.08613376 0.28499898]


# Hiperopt for the optimalized version

In [42]:
top_features = train_importance_df_pca['Feature'].head(30).tolist()
X = X_train

xgb_base = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)

multi_xgb = MultiOutputRegressor(xgb_base)

param_grid = {
    'estimator__n_estimators': [50, 75, 100],
    'estimator__max_depth': [3, 5, 7],
    'estimator__learning_rate': [0.01, 0.1, 0.05]
}

grid_search = GridSearchCV(estimator=multi_xgb, param_grid=param_grid,
                           cv=2, n_jobs=-1, verbose=2, scoring='r2')

grid_search.fit(X_train, y_train)

print("Best parameters found: ", grid_search.best_params_)
print("Best R² score found: ", grid_search.best_score_)

Fitting 2 folds for each of 27 candidates, totalling 54 fits


/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()


Best parameters found:  {'estimator__learning_rate': 0.05, 'estimator__max_depth': 7, 'estimator__n_estimators': 100}
Best R² score found:  0.22157095933015739


In [43]:
top_features = train_importance_df_pca['Feature'].head(30).tolist()
X_train = X_train[top_features]
X_val = X_val[top_features]


multi_xgb_model = MultiOutputRegressor(xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, random_state=42, learning_rate=0.05, max_depth=7))

multi_xgb_model.fit(X_train, y_train)

y_pred_multi = multi_xgb_model.predict(X_val)

r2_scores = r2_score(y_val, y_pred_multi, multioutput='raw_values')
print("R² scores for each target (PCA with the top 30 important features):", r2_scores)

R² scores for each target (PCA with the top 30 important features): [0.17367598 0.22243348 0.35978511 0.26569924 0.11340901 0.29138472]


# Training on full dataset

In [65]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [66]:
all_targets = [col for col in train_df.columns if col not in test_df.columns]
targets = all_targets[:6]

In [67]:
metadata_columns = [col for col in train_df.columns if col in test_df.columns and col != 'id']

In [68]:
train_df = train_df[train_df['X4_mean'] > -1]

In [69]:
  # Apply log10 transformation to the specified columns
for column in targets:
    train_mapped_values = train_df[column] + 1

    train_df[column] = np.log10(train_mapped_values)  # Adding 1 to avoid log(0)


In [70]:
X_train, y_train = train_df[metadata_columns], train_df[targets]

In [71]:
X_test = test_df

In [44]:
xgb_model_full = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, random_state=42)
xgb_model_full.fit(X_train, y_train)

importances = xgb_model_full.feature_importances_

In [45]:
feature_names = train_df[metadata_columns].columns
feature_names.shape

(163,)

In [46]:
importances.shape

(163,)

In [47]:
train_importance_df_pca = pd.DataFrame({'Feature': feature_names, 'Importance': importances})
train_importance_df_pca = train_importance_df_pca.sort_values(by='Importance', ascending=False).reset_index(drop=True)
print(train_importance_df_pca.head(20))

                                              Feature  Importance
0              WORLDCLIM_BIO1_annual_mean_temperature    0.040546
1             WORLDCLIM_BIO7_temperature_annual_range    0.034594
2              WORLDCLIM_BIO4_temperature_seasonality    0.022595
3                   SOIL_phh2o_60.100cm_mean_0.01_deg    0.013326
4   MODIS_2000.2020_monthly_mean_surface_reflectan...    0.013121
5                     SOIL_ocd_60.100cm_mean_0.01_deg    0.011998
6   MODIS_2000.2020_monthly_mean_surface_reflectan...    0.010915
7   MODIS_2000.2020_monthly_mean_surface_reflectan...    0.010303
8                    SOIL_sand_60.100cm_mean_0.01_deg    0.010056
9   WORLDCLIM_BIO13.BIO14_delta_precipitation_of_w...    0.009926
10                      SOIL_silt_0.5cm_mean_0.01_deg    0.008869
11                 VOD_C_2002_2018_multiyear_mean_m12    0.008461
12  MODIS_2000.2020_monthly_mean_surface_reflectan...    0.008390
13  MODIS_2000.2020_monthly_mean_surface_reflectan...    0.008138
14        

In [81]:
top_features

['WORLDCLIM_BIO1_annual_mean_temperature',
 'WORLDCLIM_BIO7_temperature_annual_range',
 'WORLDCLIM_BIO4_temperature_seasonality',
 'SOIL_phh2o_60.100cm_mean_0.01_deg',
 'MODIS_2000.2020_monthly_mean_surface_reflectance_band_01_._month_m7',
 'SOIL_ocd_60.100cm_mean_0.01_deg',
 'MODIS_2000.2020_monthly_mean_surface_reflectance_band_03_._month_m4',
 'MODIS_2000.2020_monthly_mean_surface_reflectance_band_01_._month_m9',
 'SOIL_sand_60.100cm_mean_0.01_deg',
 'WORLDCLIM_BIO13.BIO14_delta_precipitation_of_wettest_and_dryest_month',
 'SOIL_silt_0.5cm_mean_0.01_deg',
 'VOD_C_2002_2018_multiyear_mean_m12',
 'MODIS_2000.2020_monthly_mean_surface_reflectance_band_01_._month_m8',
 'MODIS_2000.2020_monthly_mean_surface_reflectance_band_03_._month_m10',
 'SOIL_sand_0.5cm_mean_0.01_deg',
 'MODIS_2000.2020_monthly_mean_surface_reflectance_band_01_._month_m10',
 'MODIS_2000.2020_monthly_mean_surface_reflectance_band_02_._month_m8',
 'VOD_Ku_1987_2017_multiyear_mean_m07',
 'SOIL_silt_30.60cm_mean_0.01_de

In [73]:
top_features = train_importance_df_pca['Feature'].head(30).tolist()
X_train = X_train[top_features]
X_val = X_test[top_features]


multi_xgb_model = MultiOutputRegressor(xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, random_state=42, learning_rate=0.05, max_depth=7))

multi_xgb_model.fit(X_train, y_train)


MultiOutputRegressor(estimator=XGBRegressor(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None, device=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None,
                                            feature_types=None, gamma=None,
                                            grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=0.05, max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=7,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=100, n_jobs=None,
                                            num_parallel_tree=None,
                                            random_state=42, ...))

In [74]:
y_pred_multi = multi_xgb_model.predict(X_test[top_features])

print("Pred for each target (PCA with the top 30 important features):", y_pred_multi)

Pred for each target (PCA with the top 30 important features): [[0.18532589 1.068016   0.38652593 0.47135115 0.39616558 2.4421654 ]
 [0.16374885 1.2704012  0.16314845 0.5323429  0.39608303 2.6481786 ]
 [0.15812114 1.1967387  0.19328308 0.47708488 0.4075253  2.584163  ]
 ...
 [0.16381808 1.3204018  0.27360857 0.5468797  0.34060335 2.995404  ]
 [0.15722279 1.2826252  0.23528276 0.58774775 0.37981167 2.8633046 ]
 [0.1972119  1.1438962  0.8975595  1.5863138  0.41266456 3.300521  ]]


Prepara data for submission

In [75]:
targets

['X4_mean', 'X11_mean', 'X18_mean', 'X26_mean', 'X50_mean', 'X3112_mean']

In [76]:
df = pd.DataFrame(y_pred_multi, columns=targets)

In [77]:
original_values = 10 ** df[targets] - 1

In [78]:
original_values['id'] = test_df['id']

In [79]:
original_values

,X4_mean,X11_mean,X18_mean,X26_mean,X50_mean,X3112_mean,id
0,0.532237,10.695426,1.435151,1.960405,1.489806,275.799561,201238668
1,0.457971,17.638083,0.455957,2.406771,1.489333,443.814148,202310319
2,0.439200,14.730362,0.560569,1.999749,1.555791,382.851257,202604412
3,0.425343,20.435699,0.412526,1.517635,1.288963,594.578552,201353439
4,0.486598,10.830256,0.434011,1.239148,1.510673,219.854233,195351745
...,...,...,...,...,...,...,...
6540,0.662212,8.517436,1.709337,6.851376,1.766796,212.315964,195548469
6541,0.526767,15.820763,7.011061,35.304901,1.484938,6347.016602,199261251
6542,0.458203,19.912300,0.877624,2.522733,1.190803,988.473145,203031744
6543,0.436226,18.170135,0.719027,2.870328,1.397793,728.969360,197736382


In [58]:
targets.insert(0,'id')

In [59]:
targets

['id', 'X4_mean', 'X11_mean', 'X18_mean', 'X26_mean', 'X50_mean', 'X3112_mean']

In [60]:
original_values

,X4_mean,X11_mean,X18_mean,X26_mean,X50_mean,X3112_mean,id
0,0.533318,10.307443,1.351844,2.278717,1.605068,236.653320,201238668
1,0.445463,16.786470,0.469525,2.340830,1.528038,382.681641,202310319
2,0.440525,15.288557,0.564903,1.867717,1.590743,378.354340,202604412
3,0.425560,20.423512,0.417129,1.559785,1.268768,618.828918,201353439
4,0.495263,11.296143,0.420331,1.616960,1.531291,199.023239,195351745
...,...,...,...,...,...,...,...
6540,0.656417,8.856338,1.889847,6.468963,1.834585,214.010269,195548469
6541,0.527181,15.415337,6.926759,18.759579,1.479276,6661.014160,199261251
6542,0.458035,19.924978,0.851065,2.471447,1.197199,993.317261,203031744
6543,0.438335,17.933077,0.842907,2.894757,1.409507,714.739807,197736382


In [61]:
result = original_values[['id', 'X4_mean', 'X11_mean', 'X18_mean', 'X50_mean', 'X26_mean', 'X3112_mean']]

In [62]:
result.columns = ['id', 'X4', 'X11', 'X18', 'X50', 'X26', 'X3112']

In [63]:
result

,id,X4,X11,X18,X50,X26,X3112
0,201238668,0.533318,10.307443,1.351844,1.605068,2.278717,236.653320
1,202310319,0.445463,16.786470,0.469525,1.528038,2.340830,382.681641
2,202604412,0.440525,15.288557,0.564903,1.590743,1.867717,378.354340
3,201353439,0.425560,20.423512,0.417129,1.268768,1.559785,618.828918
4,195351745,0.495263,11.296143,0.420331,1.531291,1.616960,199.023239
...,...,...,...,...,...,...,...
6540,195548469,0.656417,8.856338,1.889847,1.834585,6.468963,214.010269
6541,199261251,0.527181,15.415337,6.926759,1.479276,18.759579,6661.014160
6542,203031744,0.458035,19.924978,0.851065,1.197199,2.471447,993.317261
6543,197736382,0.438335,17.933077,0.842907,1.409507,2.894757,714.739807


In [64]:
result.to_csv('submission.csv',index=False)

In [ ]:
pd.read_csv('/content/submission.csv')

,id,X4,X11,X18,X26,X50,X3112
0,201238668,0.580484,9.217249,2.216736,2.095231,1.943165,219.01149
1,202310319,0.454179,17.287480,0.558951,2.676731,1.497181,391.16614
2,202604412,0.441764,14.351482,0.557930,2.116096,1.683865,370.00810
3,201353439,0.447919,19.619755,0.377674,1.590339,1.226824,608.13890
4,195351745,0.477700,11.745309,0.372706,1.275382,1.551120,184.64620
...,...,...,...,...,...,...,...
6540,195548469,0.599329,10.515012,2.227373,4.972947,1.677731,286.04257
6541,199261251,0.537146,14.797981,6.472179,38.069890,1.489934,5057.75500
6542,203031744,0.461706,19.150640,0.945556,2.300021,1.211896,793.44574
6543,197736382,0.448688,18.046192,0.650641,3.837817,1.391281,735.29500


In [ ]:
print("Pred for each target (PCA with the top 30 important features):", y_pred_multi.shape)

In [ ]:
sample = pd.read_csv('/content/sample_submission.csv')

In [ ]:
sample

,id,X4,X11,X18,X50,X26,X3112
0,201238668,21.228750,46.704747,31.430122,14.088638,24.208952,93.351051
1,202310319,4.938497,71.644437,50.208404,99.503446,86.561125,20.325735
2,202604412,34.268159,93.046483,74.556941,84.275746,85.713291,23.470287
3,201353439,48.213879,84.687775,-1.442158,-9.852137,13.327949,-0.274766
4,195351745,55.117501,87.046172,84.335483,79.218376,19.142174,11.294033


# TAB TRANSFORMER

In [ ]:

# Separate features and targets
X_train = train_df[metadata_columns]  # Adjust based on your target
Y_train = train_df[targets]
X_val = val_df[metadata_columns]
Y_val = val_df[targets]



In [ ]:
!pip install torchmetrics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 841.5/841.5 kB 5.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
from torchmetrics.functional import r2_score

### note: The kernel will restrart after this at the first time

In [ ]:
pip install -U pytorch_tabular

In [ ]:
celvaltozok = targets
bemeno_valtozok = metadata_columns

# 1. modell

In [ ]:
from pytorch_tabular import TabularModel
from pytorch_tabular.models import TabTransformerConfig
from pytorch_tabular.config import (
    DataConfig,
    OptimizerConfig,
    TrainerConfig,
)

data_config = DataConfig(
    target= celvaltozok,  # target should always be a list. Multi-targets are only supported for regression. Multi-Task Classification is not implemented
    continuous_cols=bemeno_valtozok,
    categorical_cols=[],
)

# Define the training configuration
trainer_config = TrainerConfig(
    auto_lr_find=True,  # Automatically find a learning rate
    batch_size=128,
    max_epochs=100,
)

# Define the optimizer configuration
optimizer_config = OptimizerConfig()

# Define the TabTransformer model configuration for regression
model_config = TabTransformerConfig(
    task="regression",
    embedding_dropout=0.1,  # Dropout for embeddings
    num_attn_blocks=2,  # Number of attention blocks
    num_heads=4,  # Number of attention heads
    transformer_activation="GEGLU",
    learning_rate = 0.01 # Activation function
    # You can define other hyperparameters as needed
)

# Create the TabularModel with the defined configurations
tabular_model = TabularModel(
    data_config=data_config,
    model_config=model_config,
    optimizer_config=optimizer_config,
    trainer_config=trainer_config,
)


INFO:pytorch_tabular.tabular_model:Experiment Tracking is turned off


In [ ]:
tabular_model.fit(train=train_df, validation=val_df)
result = tabular_model.evaluate(val_df)
pred_df = tabular_model.predict(val_df)
tabular_model.save_model("examples/basic")
loaded_model = TabularModel.load_model("examples/basic")

INFO:lightning_fabric.utilities.seed:Seed set to 42
INFO:pytorch_tabular.tabular_model:Preparing the DataLoaders
INFO:pytorch_tabular.tabular_datamodule:Setting up the datamodule for regression task
INFO:pytorch_tabular.tabular_model:Preparing the Model: TabTransformerModel
INFO:pytorch_tabular.tabular_model:Preparing the Trainer
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_tabular.tabular_model:Auto LR Find Started
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:639: Checkpoint directory /content/saved_models exists and is not empty.
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Finding best initial lr:   0%|          | 0/100 [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_steps=100` reached.
INFO:pytorch_lightning.tuner.lr_finder:Learning rate set to 0.036307805477010104
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at /content/.lr_find_0e6a7ee8-7b96-4201-a55e-71ba57c52703.ckpt
INFO:pytorch_lightning.utilities.rank_zero:Restored all states from the checkpoint at /content/.lr_find_0e6a7ee8-7b96-4201-a55e-71ba57c52703.ckpt
INFO:pytorch_tabular.tabular_model:Suggested LR: 0.036307805477010104. For plot and detailed analysis, use `find_learning_rate` method.
INFO:pytorch_tabular.tabular_model:Training Started
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name             ┃ Type                   ┃ Params ┃
┡━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ _backbone        │ TabTransformerBackbone │ 90.8 K │
│ 1 │ _embedding_layer │ Embedding2dLayer       │      0 │
│ 2 │ _head            │ LinearHead             │    984 │
│ 3 │ loss             │ MSELoss                │      0 │
└───┴──────────────────┴────────────────────────┴────────┘

Trainable params: 91.7 K                                                                                           
Non-trainable params: 0                                                                                            
Total params: 91.7 K                                                                                               
Total estimated model params size (MB): 0

Output()

INFO:pytorch_tabular.tabular_model:Training the model completed
INFO:pytorch_tabular.tabular_model:Loading the best model
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Output()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │    1.0362744331359863     │
│        test_loss_0        │   0.001999471802264452    │
│        test_loss_1        │    0.04827537387609482    │
│        test_loss_2        │    0.12170775234699249    │
│        test_loss_3        │    0.47391143441200256    │
│        test_loss_4        │   0.014380091801285744    │
│        test_loss_5        │    0.3760002851486206     │
│  test_mean_squared_error  │    1.0362744331359863     │
│ test_mean_squared_error_0 │   0.001999471802264452    │
│ test_mean_squared_error_1 │    0.04827537387609482    │
│ test_mean_squared_error_2 │    0.12170775234699249    │
│ test_mean_squared_error_3 │    0.47391143441200256    │
│ test_mean_squared_error_4 │   0.014380091801285744    │
│ test_mean_squared_error_5 │    0.3760002851486206     │
└───────────────────────────┴───────────────────────────┘

INFO:pytorch_tabular.tabular_model:Experiment Tracking is turned off
INFO:pytorch_tabular.tabular_model:Preparing the Trainer
INFO:pytorch_lightning.utilities.rank_zero:Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.rich_model_summary.RichModelSummary'>]. Skipping setting a default `ModelSummary` callback.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
r2_score(torch.tensor(y_val.values), torch.tensor(pred_df.values))

tensor(-3.3028, dtype=torch.float64)

#2. modell

In [ ]:
from pytorch_tabular import TabularModel
from pytorch_tabular.models import CategoryEmbeddingModelConfig
from pytorch_tabular.config import (
    DataConfig,
    OptimizerConfig,
    TrainerConfig,
    ExperimentConfig,
)

data_config = DataConfig(
    target= celvaltozok,  # target should always be a list. Multi-targets are only supported for regression. Multi-Task Classification is not implemented
    continuous_cols=bemeno_valtozok,
    categorical_cols=[],
)
trainer_config = TrainerConfig(
    auto_lr_find=True,  # Runs the LRFinder to automatically derive a learning rate
    batch_size=64,
    max_epochs=100,
)
optimizer_config = OptimizerConfig()

model_config = CategoryEmbeddingModelConfig(
    task="regression",
    layers="1024-512-256",  # Number of nodes in each layer
    activation="LeakyReLU",  # Activation between each layers
    learning_rate=1e-3,
    dropout=0.1,
    #use_batch_norm=True,
    #loss = r2_loss,
)

tabular_model = TabularModel(
    data_config=data_config,
    model_config=model_config,
    optimizer_config=optimizer_config,
    trainer_config=trainer_config,
)


INFO:pytorch_tabular.tabular_model:Experiment Tracking is turned off


In [ ]:
tabular_model.fit(train=train_df, validation=val_df)
result = tabular_model.evaluate(val_df)
pred_df = tabular_model.predict(val_df)
tabular_model.save_model("examples/basic")
loaded_model = TabularModel.load_model("examples/basic")

INFO:lightning_fabric.utilities.seed:Seed set to 42
INFO:pytorch_tabular.tabular_model:Preparing the DataLoaders
INFO:pytorch_tabular.tabular_datamodule:Setting up the datamodule for regression task
INFO:pytorch_tabular.tabular_model:Preparing the Model: CategoryEmbeddingModel
INFO:pytorch_tabular.tabular_model:Preparing the Trainer
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_tabular.tabular_model:Auto LR Find Started
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:639: Checkpoint directory /content/saved_models exists and is not empty.
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Finding best initial lr:   0%|          | 0/100 [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Trainer was signaled to stop but the required `min_epochs=1` or `min_steps=None` has not been met. Training will continue...
INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_steps=100` reached.
INFO:pytorch_lightning.tuner.lr_finder:Learning rate set to 0.5754399373371567
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at /content/.lr_find_8c4d7b6f-4f14-4180-bd67-62b782671560.ckpt
INFO:pytorch_lightning.utilities.rank_zero:Restored all states from the checkpoint at /content/.lr_find_8c4d7b6f-4f14-4180-bd67-62b782671560.ckpt
INFO:pytorch_tabular.tabular_model:Suggested LR: 0.5754399373371567. For plot and detailed analysis, use `find_learning_rate` method.
INFO:pytorch_tabular.tabular_model:Training Started
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name             ┃ Type                      ┃ Params ┃
┡━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ _backbone        │ CategoryEmbeddingBackbone │  824 K │
│ 1 │ _embedding_layer │ Embedding1dLayer          │    326 │
│ 2 │ head             │ LinearHead                │  1.5 K │
│ 3 │ loss             │ MSELoss                   │      0 │
└───┴──────────────────┴───────────────────────────┴────────┘

Trainable params: 825 K                                                                                            
Non-trainable params: 0                                                                                            
Total params: 825 K                                                                                                
Total estimated model params size (MB): 3

Output()

INFO:pytorch_tabular.tabular_model:Training the model completed
INFO:pytorch_tabular.tabular_model:Loading the best model
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Output()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │        922107520.0        │
│        test_loss_0        │        154810336.0        │
│        test_loss_1        │        150183936.0        │
│        test_loss_2        │        168709616.0        │
│        test_loss_3        │        145587200.0        │
│        test_loss_4        │        148325008.0        │
│        test_loss_5        │        154491632.0        │
│  test_mean_squared_error  │        922107520.0        │
│ test_mean_squared_error_0 │        154810336.0        │
│ test_mean_squared_error_1 │        150183936.0        │
│ test_mean_squared_error_2 │        168709616.0        │
│ test_mean_squared_error_3 │        145587200.0        │
│ test_mean_squared_error_4 │        148325008.0        │
│ test_mean_squared_error_5 │        154491632.0        │
└───────────────────────────┴───────────────────────────┘

INFO:pytorch_tabular.tabular_model:Experiment Tracking is turned off
INFO:pytorch_tabular.tabular_model:Preparing the Trainer
INFO:pytorch_lightning.utilities.rank_zero:Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.rich_model_summary.RichModelSummary'>]. Skipping setting a default `ModelSummary` callback.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
r2_score(torch.tensor(y_val.values), torch.tensor(pred_df.values))

tensor(-2.5783, dtype=torch.float64)

In [ ]:
for i in range(5):
    print(f"feature {i} r2 score: {r2_score(torch.tensor(y_val.values[:,i]), torch.tensor(pred_df.values[:,i]))}")

feature 0 r2 score: -2.971248001814826
feature 1 r2 score: -2.751134704352312
feature 2 r2 score: -2.7750289605572354
feature 3 r2 score: -2.045008208545667
feature 4 r2 score: -2.4265495081629362


In [ ]:
from pytorch_tabular import TabularModel
from pytorch_tabular.models import CategoryEmbeddingModelConfig
from pytorch_tabular.config import (
    DataConfig,
    OptimizerConfig,
    TrainerConfig,
    ExperimentConfig,
)

data_config = DataConfig(
    target= celvaltozok,  # target should always be a list. Multi-targets are only supported for regression. Multi-Task Classification is not implemented
    continuous_cols=bemeno_valtozok,
    categorical_cols=[],
)
trainer_config = TrainerConfig(
    auto_lr_find=True,  # Runs the LRFinder to automatically derive a learning rate
    batch_size=64,
    max_epochs=100,
)
optimizer_config = OptimizerConfig()

model_config = CategoryEmbeddingModelConfig(
    task="regression",
    layers="1024-512-128",  # Number of nodes in each layer
    activation="LeakyReLU",  # Activation between each layers
    learning_rate=1e-3,
    dropout=0.1,
    batch_norm_continuous_input = False,
    #use_batch_norm=True,
    #loss = r2_loss,
)

tabular_model = TabularModel(
    data_config=data_config,
    model_config=model_config,
    optimizer_config=optimizer_config,
    trainer_config=trainer_config,
)

tabular_model.fit(train=train_df, validation=val_df)
result = tabular_model.evaluate(val_df)
pred_df = tabular_model.predict(val_df)
tabular_model.save_model("examples/basic")
loaded_model = TabularModel.load_model("examples/basic")

r2_score(torch.tensor(y_val.values), torch.tensor(pred_df.values))

INFO:pytorch_tabular.tabular_model:Experiment Tracking is turned off
INFO:lightning_fabric.utilities.seed:Seed set to 42
INFO:pytorch_tabular.tabular_model:Preparing the DataLoaders
INFO:pytorch_tabular.tabular_datamodule:Setting up the datamodule for regression task
INFO:pytorch_tabular.tabular_model:Preparing the Model: CategoryEmbeddingModel
INFO:pytorch_tabular.tabular_model:Preparing the Trainer
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_tabular.tabular_model:Auto LR Find Started
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:639: Checkpoint directory /content/saved_models exists and is not empty.
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 

Finding best initial lr:   0%|          | 0/100 [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Trainer was signaled to stop but the required `min_epochs=1` or `min_steps=None` has not been met. Training will continue...
INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_steps=100` reached.
INFO:pytorch_lightning.tuner.lr_finder:Learning rate set to 0.5754399373371567
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at /content/.lr_find_51ecb45f-5ff5-4f25-852f-6d60b6f1fc10.ckpt
INFO:pytorch_lightning.utilities.rank_zero:Restored all states from the checkpoint at /content/.lr_find_51ecb45f-5ff5-4f25-852f-6d60b6f1fc10.ckpt
INFO:pytorch_tabular.tabular_model:Suggested LR: 0.5754399373371567. For plot and detailed analysis, use `find_learning_rate` method.
INFO:pytorch_tabular.tabular_model:Training Started
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name             ┃ Type                      ┃ Params ┃
┡━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ _backbone        │ CategoryEmbeddingBackbone │  824 K │
│ 1 │ _embedding_layer │ Embedding1dLayer          │      0 │
│ 2 │ head             │ LinearHead                │  1.5 K │
│ 3 │ loss             │ MSELoss                   │      0 │
└───┴──────────────────┴───────────────────────────┴────────┘

Trainable params: 825 K                                                                                            
Non-trainable params: 0                                                                                            
Total params: 825 K                                                                                                
Total estimated model params size (MB): 3

...

INFO:pytorch_tabular.tabular_model:Training the model completed
INFO:pytorch_tabular.tabular_model:Loading the best model
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Output()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │       25207244800.0       │
│        test_loss_0        │       4396463104.0        │
│        test_loss_1        │       4584890880.0        │
│        test_loss_2        │       4141804800.0        │
│        test_loss_3        │       4161140480.0        │
│        test_loss_4        │       3557924352.0        │
│        test_loss_5        │       4365014528.0        │
│  test_mean_squared_error  │       25207244800.0       │
│ test_mean_squared_error_0 │       4396463104.0        │
│ test_mean_squared_error_1 │       4584890880.0        │
│ test_mean_squared_error_2 │       4141804800.0        │
│ test_mean_squared_error_3 │       4161140480.0        │
│ test_mean_squared_error_4 │       3557924352.0        │
│ test_mean_squared_error_5 │       4365014528.0        │
└───────────────────────────┴───────────────────────────┘

INFO:pytorch_tabular.tabular_model:Experiment Tracking is turned off
INFO:pytorch_tabular.tabular_model:Preparing the Trainer
INFO:pytorch_lightning.utilities.rank_zero:Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.rich_model_summary.RichModelSummary'>]. Skipping setting a default `ModelSummary` callback.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


tensor(-2.1676, dtype=torch.float64)